In [ ]:
import os
import json
import logging
import numpy as np

In [ ]:
def load_json_from_lib(nome_file, local = False):
    # Usa __file__ per ottenere il percorso della directory del file corrente
    if not local:
        file_path = os.path.join(os.path.dirname(__file__), nome_file)
    else:
        file_path = nome_file
    with open(file_path, 'r') as file:
        return json.load(file)

In [ ]:
import sys
import os

# Append the parent directory to sys.path
sys.path.append(os.path.abspath('..'))

# Now you can import the module
from llm_utils import api_keys
api_keys

In [ ]:
print("\n\n<<<<<<<<<<<<< No-RAG Reply >>>>>>>>>>>>>>>")
GROQ_API_KEY = api_keys["groq"]
MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
# MODEL = "llama-3.1-8b-instant"

# Test Native LLM response
from groq import Groq

client = Groq(api_key=GROQ_API_KEY)
question = "What is the capital of France?"

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": question},
    ],
    stream=False
)

print(response.choices[0].message.content)
print("<<< ----------------- >>>")

# Data Preparation

In [ ]:
!pip install groq

In [ ]:
import pandas as pd

dataset_df = pd.read_csv("../datasets/working_dataset_1000.csv")
dataset_df

In [ ]:
dataset_df['TOPIC_LABEL'].unique()

In [ ]:
# df_FOODAL = dataset_df[dataset_df["TOPIC_LABEL"] == ["FOODAL"]]
# df_OBWCCE = dataset_df[dataset_df["TOPIC_LABEL"] == ["OBWCCE"]]
# df_EATBTS = dataset_df[dataset_df["TOPIC_LABEL"] == ["EATBTS"]]

df_FOODAL = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'FOODAL' in x)]
df_OBWCCE = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'OBWCCE' in x)]
df_EATBTS = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'EATBTS' in x)]
df_VITMIN = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'VITMIN' in x)]
df_FOODIN = dataset_df[dataset_df['TOPIC_LABEL'].apply(lambda x: 'FOODIN' in x)]

In [ ]:
working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"
df_FOODAL.sample(5)['RESULTS'].to_json(working_dir+"foodal.json", orient="records")
df_OBWCCE.sample(5)['RESULTS'].to_json(working_dir+"obwcce.json", orient="records")
df_EATBTS.sample(5)['RESULTS'].to_json(working_dir+"eatbts.json", orient="records")
df_VITMIN.sample(5)['RESULTS'].to_json(working_dir+"vitmin.json", orient="records")
df_FOODIN.sample(5)['RESULTS'].to_json(working_dir+"foodin.json", orient="records")

In [ ]:
# prompt: read a json file and count the element
import json

working_dir = os.getcwd() + "/documents_from_working_dataset/"
file_path = working_dir+"foodal.json"  # Replace with the actual path to your JSON file

with open(file_path, 'r') as file:
  data = json.load(file)

# Assuming the JSON file contains a list or a dictionary
if isinstance(data, list):
  element_count = len(data)
  print(f"The JSON file contains {element_count} elements in the list.")
elif isinstance(data, dict):
  element_count = len(data)
  print(f"The JSON file contains {element_count} key-value pairs in the dictionary.")
else:
  print("The JSON file does not contain a list or a dictionary at the top level.")

In [ ]:
role_PROMPT = """
      You are a helpful assistant responsible for generating evaluation questions
      from a given corpus using a structured procedure."""
goal_PROMPT = """
      Given a corpus description and the parameters K (number of users), N (number of tasks per user),
      and M (number of high-level questions per (user, task) pair), your task is to generate high-level questions
      that assess global understanding of the corpus.
      """
generation_step_PROMPT = """
      Your generation should follow these steps:
      1. Create K user personas based on the corpus, including their role/background and motivation for using the corpus.
      2. For each user, define N realistic and relevant tasks they would perform using the corpus.
      3. For each (user, task) pair, generate M high-level questions that:
          - Require comprehensive understanding of the whole corpus.
          - Do NOT depend on low-level facts or specific data points (e.g., names, dates, figures)."""

output_PROMPT = """
      User 1: [Persona description]
      Task 1:
      Q1. ...
      Q2. ...
      ...
      Task 2:
      Q1. ...
      ...
      User 2: ...
      ...
      User K: ..."""

output_format_PROMPT = """
      {{
        "corpus_description": "<your summary of the corpus>",
        "generated_questions": "<your structured output as described above>"
      }}"""

In [ ]:
def generate_questions(corpus_description, K, N, M):
  PROMPT = f"""
  ---Role--
  {role_PROMPT}

  ---Goal--
  {goal_PROMPT}

  {generation_step_PROMPT}

  Your output should be structured as follows:
  {output_PROMPT}

  Format your response as a JSON object with the following structure:
  {output_format_PROMPT}

  ---Corpus Description--
  {corpus_description}
  ---Parameters--
  Number of Users (K): {K}
  Tasks per User (N): {N}
  Questions per (User, Task) (M): {M}

  Output:
  """
  return PROMPT

In [ ]:
import os
from groq import Groq

MODEL = "meta-llama/llama-4-scout-17b-16e-instruct"
GROQ_API_KEY = "***"
GROQ_API_KEY = api_keys["groq"]

client = Groq(
    api_key=GROQ_API_KEY,
)

In [ ]:
with open('/content/drive/MyDrive/workshop-ECAI/groq_api.txt', 'w') as f:
  f.write(GROQ_API_KEY)

In [ ]:
working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"
foodal_docs = json.load(open(working_dir+"foodal.json"))
obwcce_docs = json.load(open(working_dir+"obwcce.json"))
eatbts_docs = json.load(open(working_dir+"eatbts.json"))
vitmin_docs = json.load(open(working_dir+"vitmin.json"))
foodin_docs = json.load(open(working_dir+"foodin.json"))

In [ ]:
corpus_description_PROMPT = f"""
---Role--
You are a helpful assistant tasked with generating a description of a corpus based on the results of 5 randomly selected academic papers.

---Goal--
Given a set of 5 academic papers, your task is to summarize the key findings from each paper and combine this information into a coherent corpus description. The description should highlight common themes, methodologies used, and major conclusions drawn across the papers.

---Papers---
1. {foodin_docs[0]}
2. {foodin_docs[1]}
3. {foodin_docs[2]}
4. {foodin_docs[3]}
5. {foodin_docs[4]}

---Instructions---
For each paper:
- Briefly summarize the study's objective.
- Describe the methodology used.
- List the main findings or results.

After analyzing all 5 papers:
- Identify common themes across the papers.
- Highlight any significant differences in methodologies or results.
- Provide an overall description of the corpus that reflects the combined knowledge and insights from the papers.

---Output Format---
Return your response as a single text object with the following structure:

<Detailed description of the corpus based on the 5 papers>

---Output---

"""


chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": corpus_description_PROMPT
        }
    ],
    model=MODEL,
)

answer = chat_completion.choices[0].message.content

print(answer)

In [ ]:
# prompt: write the foodal_answer in a file text in my folder in drive

corpus_description = answer.split("SNVs, indels, and copy number variants.\n\n")[1]
print(corpus_description)

with open(working_dir+'foodin_answer.txt', 'w') as f:
  f.write(corpus_description)

# Question Generation

In [ ]:
K = 3
N = 5
M = 5
working_dir = "/content/drive/MyDrive/workshop-ECAI/"
working_dir = os.getcwd() + "/documents_from_working_dataset/"
with open(working_dir+'foodin_answer.txt', 'r') as file:
  corpus_description = file.read()

prompt = generate_questions(corpus_description, K, N, M)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    model=MODEL,
)

answer = chat_completion.choices[0].message.content

print(answer)

In [ ]:
question_dir = os.getcwd() + "/questions_files/"
with open(question_dir+'foodin_generated_question.txt', 'w') as f:
  f.write(answer)

# Restructuring questions in a DataFrame


In [ ]:
question_dir = os.getcwd() + "/questions_files/"

with open(question_dir+'foodal_generated_question.txt', 'r') as file:
  foodal_questions = file.read()

with open(question_dir+'obwcce_generated_question.txt', 'r') as file:
  obwcce_questions = file.read()

with open(question_dir+'eatbts_generated_question.txt', 'r') as file:
  eatbts_questions = file.read()

with open(question_dir+'vitmin_generated_question.txt', 'r') as file:
  vitmin_questions = file.read()

with open(question_dir+'foodin_generated_question.txt', 'r') as file:
  foodin_questions = file.read()

In [ ]:
print(foodal_questions)

In [ ]:
import pandas as pd
import re

# Step 1: Read file as plain text
#-- Fatto nella cella sopra per tutti

questions = [foodin_questions, foodal_questions, obwcce_questions, eatbts_questions, vitmin_questions]
categories = ["FOODIN", "FOODAL", "OBWCCE", "EATBTS", "VITMIN"]

data = {}

for i, q in enumerate(questions):

    # Step 2: Manually extract the 'generated_questions' content
    match = re.search(r'"generated_questions"\s*:\s*"(?P<content>.*)"\s*}', q, re.DOTALL)
    if not match:
        raise ValueError("Could not find 'generated_questions' content in the file.")

    content = match.group("content")

    # Step 3: Unescape escaped quotes and normalize the content
    content = content.replace('\\"', '"')

    # Step 4: Parse lines
    lines = content.splitlines()


    category = categories[i]  # or derive from file name
    print(f"Processing category: {category}")

    rows = []
    current_user = None
    current_task = None
    questions = []

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith("User"):
            current_user = line.split(":", 1)[1].strip()
        elif line.startswith("Task"):
            if current_task and questions:
                rows.append({
                    "Category": category,
                    "USER Description": current_user,
                    "TASK Description": current_task,
                    "Questions": questions
                })
                questions = []
            current_task = line.split(":", 1)[1].strip()
        elif re.match(r"Q\d+\.", line):
            questions.append(line)

    # Append the last block
    if current_task and questions:
        rows.append({
            "Category": category,
            "USER Description": current_user,
            "TASK Description": current_task,
            "Questions": questions
        })
    
    data[category] = rows



In [ ]:
# Step 5: Create DataFrame
df_foodal = pd.DataFrame(data["FOODAL"])
df_obwcce = pd.DataFrame(data["OBWCCE"])
df_eatbts = pd.DataFrame(data["EATBTS"])
df_vitmin = pd.DataFrame(data["VITMIN"])
df_foodin = pd.DataFrame(data["FOODIN"])
df_foodal

In [ ]:
# prompt: append all these dataframe together: df_foodal

df_combined = pd.concat([df_foodal, df_obwcce, df_eatbts, df_vitmin, df_foodin], ignore_index=True)

In [44]:

df_combined.to_pickle("questions_dataframe.pkl")
df_combined

,Category,USER Description,TASK Description,Questions
0,FOODAL,A clinical researcher interested in vaccine de...,Designing a new vaccine trial,[Q1. What are the key genetic and immunologica...
1,FOODAL,A clinical researcher interested in vaccine de...,Analyzing genetic data from a new study,[Q1. How do the genetic associations with dise...
2,FOODAL,A clinical researcher interested in vaccine de...,Developing a new therapeutic approach,[Q1. What are the key insights from the corpus...
3,FOODAL,A clinical researcher interested in vaccine de...,Communicating with patients about genetic risks,[Q1. How can I effectively communicate genetic...
4,FOODAL,A genetic epidemiologist interested in underst...,Identifying research gaps,[Q1. What are the key research gaps in our und...
...,...,...,...,...
70,FOODIN,A bioinformatics specialist analyzing large ge...,Analyzing large genetic datasets to identify d...,[Q1. How can bioinformatics tools be used to a...
71,FOODIN,A bioinformatics specialist analyzing large ge...,Investigating the genetic basis of disease usi...,[Q1. How can machine learning techniques be us...
72,FOODIN,A bioinformatics specialist analyzing large ge...,Exploring the genetic basis of complex diseases.,[Q1. How do genetic variants contribute to com...
73,FOODIN,A bioinformatics specialist analyzing large ge...,Developing computational tools for genetic ana...,[Q1. How can computational tools be used to an...


In [43]:
for desc in df_combined['USER Description'].drop_duplicates():
    print(desc,"\n------")

A clinical researcher interested in vaccine development and personalized medicine. 
------
A genetic epidemiologist interested in understanding the complex interactions between genetic and environmental factors. 
------
A toxicologist interested in understanding the impact of environmental toxins on human health. 
------
A researcher in the field of nutrition and genetics, interested in developing personalized dietary recommendations for patients with metabolic disorders. 
------
A clinician working in the field of endocrinology, interested in staying up-to-date with the latest research on metabolic disorders. 
------
A patient with a family history of metabolic disorders, interested in understanding their risk and developing a prevention plan. 
------
A healthcare researcher interested in exploring the role of genetics in disease prevention. 
------
A registered dietitian interested in developing evidence-based nutrition recommendations for patients. 
------
A healthcare policy maker 